In [1]:
import pandas as pd
import os
import numpy as np

# Get Data

In [2]:
def compare_columns(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    # Find column names in df1 but not in df2
    diff1 = columns_df1 - columns_df2
    print("Columns in df1 but not in df2:", diff1)

    # Find column names in df2 but not in df1
    diff2 = columns_df2 - columns_df1
    print("Columns in df2 but not in df1:", diff2)

In [3]:
def add_dataframes(df1, df2):
    if df1.empty:
        return df2
    if df2.empty:
        return df1
    
    # Keep only the columns that both DataFrames have
    common_columns = df1.columns.intersection(df2.columns)
    df1 = df1[common_columns]
    df2 = df2[common_columns]
    
    # Align the DataFrames
    df1, df2 = df1.align(df2, axis=1)

    # Append DataFrames
    df_sum = df1.append(df2)
    return df_sum

In [4]:
def get_folders(rootDir):
    folder_names = []
    # Specify the directory you want to start from
    rootDir = rootDir

    for name in os.listdir(rootDir):
        if os.path.isdir(os.path.join(rootDir, name)):
            folder_names.append(name)

    return folder_names

In [5]:
def get_files(folder):
    folder_path = 'Soccer Data/' + folder

    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Filter the list to only include CSV files
    csv_files = [folder + '/' + f for f in files if f.endswith('.csv')]
    return csv_files
    
#print(get_files('Spain'))
#print(get_files('England'))
#print(get_files('Italy'))
#print(get_files('Germany'))

In [6]:
def put_rank(rankings_map, Pts_map, GD_map):
    # Create a list of tuples (team, points, goal difference)
    teams = [(team, Pts_map[team], GD_map[team]) for team in Pts_map.keys()]

    # Sort the list by points and then goal difference, both in descending order
    teams.sort(key=lambda x: (x[1], x[2]), reverse=True)

    # Find the rank of the team
    for i, team in enumerate(teams, start=1):
        rankings_map[team[0]] = i

    return rankings_map

In [70]:
def get_league_play(df):
    teams = df['HomeTeam'].unique()
    
    MP_map = {}
    W_map = {}
    D_map = {}
    L_map = {}
    GF_map = {}
    GA_map = {}
    GD_map = {}
    Pts_map = {}
    rankings_map = {}
    ST_map = {}
    avgG = {}
    avgST = {}

    for team in teams:
        MP_map[team] = 0
        W_map[team] = 0
        D_map[team] = 0
        L_map[team] = 0
        GF_map[team] = 0
        GA_map[team] = 0
        GD_map[team] = 0
        Pts_map[team] = 0
        rankings_map[team] = 10   # median      
        
        ST_map[team] = 0
        avgG[team]= 0
        avgST[team] = 0

    for index, row in df.iterrows():
        
        home_team = row['HomeTeam']
        away_team = row['AwayTeam']
        
        df.loc[index, 'HGD'] = GD_map[home_team]
        df.loc[index, 'HPts'] = Pts_map[home_team]

        df.loc[index, 'AGD'] = GD_map[away_team]
        df.loc[index, 'APts'] = Pts_map[away_team]

        df.loc[index, 'HLP'] = rankings_map[home_team]
        df.loc[index, 'ALP'] = rankings_map[away_team]
        
        df.loc[index, 'avgHG'] = avgG[home_team]
        df.loc[index, 'avgAG'] = avgG[away_team]
                
        df.loc[index, 'avgHST'] = avgST[home_team]
        df.loc[index, 'avgAST'] = avgST[away_team]

        MP_map[home_team] += 1
        MP_map[away_team] += 1

        if row['FTR'] == 'H':
            W_map[home_team] += 1
            L_map[away_team] += 1
        elif row['FTR'] == 'D':
            D_map[home_team] += 1
            D_map[away_team] += 1
        elif row['FTR'] == 'A':
            L_map[home_team] += 1
            W_map[away_team] += 1

        GF_map[home_team] += row['FTHG']
        GA_map[away_team] += row['FTHG']

        GF_map[away_team] += row['FTAG']
        GA_map[home_team] += row['FTAG']
        
        ST_map[home_team] += row['HST'] if not np.isnan(row['HST']) else 0
        ST_map[away_team] += row['AST'] if not np.isnan(row['AST']) else 0

        GD_map[away_team] = GF_map[away_team] - GA_map[away_team]

        GD_map[home_team] = GF_map[home_team] - GA_map[home_team]

        Pts_map[home_team] = W_map[home_team] * 3 + D_map[home_team] * 1
        Pts_map[away_team] = W_map[away_team] * 3 + D_map[away_team] * 1
        
        # Average goals
        avgG[home_team]= GF_map[home_team] / MP_map[home_team] if MP_map[home_team] else 0
        avgG[away_team]= GF_map[away_team] / MP_map[away_team] if MP_map[away_team] else 0
        
        # Average shots on target
        avgST[home_team]= ST_map[home_team] / MP_map[home_team] if MP_map[home_team] else 0
        avgST[away_team]= ST_map[away_team] / MP_map[away_team] if MP_map[away_team] else 0

        rankings_map = put_rank(rankings_map, Pts_map, GD_map)

    return df

In [71]:
def get_data(csv_files):
    df = pd.DataFrame()

    for csv_file in csv_files:
        file_path = os.path.join(rootDir, csv_file)
        #print(csv_file)
        
        new_df = pd.read_csv(file_path)
        
        new_df['Date'] = pd.to_datetime(new_df['Date'], format="%d/%m/%Y").dt.strftime("%m/%d/%Y")
        new_df['Date'] = pd.to_datetime(new_df['Date'])
        
        #print(new_df.shape)
        new_df = get_league_play(new_df)
        
        df = add_dataframes(df, new_df)
        #print(df.shape)
        #df = df.append(new_df)
       
    return df

#get_data(get_files('Spain'))
#get_data(get_files('England'))
#get_data(get_files('Portugal'))
#get_data(get_files('Italy'))
#get_data(get_files('Germany'))

In [72]:
rootDir = 'Soccer Data'
folder_names = get_folders(rootDir)

df = pd.DataFrame()
for country_name in folder_names:
    #old_df = new_df.copy()
    
    files = get_files(country_name)
    country_df = get_data(files)
    
    if country_name == 'England':
        country_df.drop(['Referee'], axis=1, inplace=True)
        
    #print(df.shape)
    df = add_dataframes(df, country_df)
    
    #new_df = country_df.copy()
    #compare_columns(old_df, new_df)
    print(country_name)
    print(country_df.shape)
    print('df ' + str(df.shape))
    

England
(9533, 115)
df (9533, 115)
Spain
(3998, 115)
df (13531, 115)
Germany
(2916, 115)
df (16447, 115)
Portugal
(845, 115)
df (17292, 115)
Italy
(3629, 115)
df (20921, 115)


In [73]:
print(df.shape)
df.describe(include='all', datetime_is_numeric=True)

(20921, 115)


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HGD,HPts,AGD,APts,HLP,ALP,avgHG,avgAG,avgHST,avgAST
count,20921,20921,20921,20921,20921,20921.000000,20921.000000,20921,20920.000000,20920.000000,...,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000
unique,11,NaN,41,272,272,NaN,NaN,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,E1,NaN,15:00,Wigan,Charlton,NaN,NaN,H,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2661,NaN,5757,112,112,NaN,NaN,8984,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2021-12-22 10:59:36.081448960,NaN,NaN,NaN,1.447684,1.177190,NaN,0.647753,0.522897,...,-0.104536,25.705655,0.119784,25.865494,11.633526,11.608145,1.266114,1.282360,4.038382,4.078137
min,NaN,2019-07-26 00:00:00,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,...,-61.000000,0.000000,-60.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,2020-12-05 00:00:00,NaN,NaN,NaN,1.000000,0.000000,NaN,0.000000,0.000000,...,-7.000000,11.000000,-6.000000,11.000000,6.000000,6.000000,1.000000,1.000000,3.393939,3.416667
50%,NaN,2021-12-30 00:00:00,NaN,NaN,NaN,1.000000,1.000000,NaN,0.000000,0.000000,...,0.000000,23.000000,0.000000,23.000000,12.000000,12.000000,1.230769,1.241379,4.000000,4.000000
75%,NaN,2023-02-18 00:00:00,NaN,NaN,NaN,2.000000,2.000000,NaN,1.000000,1.000000,...,6.000000,38.000000,6.000000,38.000000,17.000000,17.000000,1.538462,1.550000,4.689655,4.727273
max,NaN,2024-03-18 00:00:00,NaN,NaN,NaN,9.000000,9.000000,NaN,7.000000,7.000000,...,72.000000,98.000000,72.000000,98.000000,24.000000,24.000000,6.000000,8.000000,14.000000,15.000000


In [16]:
# Select non-numerical columns
non_numerical_columns = df.select_dtypes(exclude=['int64', 'float64'])

# Print the non-numerical columns
print(non_numerical_columns.columns)

Index(['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTR', 'HTR'], dtype='object')


# Clean Data

In [74]:
df_backup = df.copy()
df_backup.shape

(20921, 115)

In [75]:
df = df_backup.copy()
df.shape

(20921, 115)

### Remove Date and Time

In [268]:
#df.drop(['Date', 'Time'], axis=1, inplace=True)
#df.shape

(20921, 103)

### Remove Cards, Fouls, and Individual Odds

In [76]:
df.drop(['Time','HF','AF','HC','AC','HY','AY','HR','AR','B365H','B365D','B365A','BWH','BWD','BWA',
 'IWH','IWD','IWA','PSH','PSD','PSA','WHH','WHD','WHA','VCH','VCD','VCA','MaxH','MaxD',
 'MaxA','B365>2.5','B365<2.5','P>2.5','P<2.5','Max>2.5','Max<2.5','B365AHH','B365AHA',
 'PAHH','PAHA','MaxAHH','MaxAHA','B365CH','B365CD','B365CA','BWCH','BWCD','BWCA','IWCH',
 'IWCD','IWCA','PSCH','PSCD','PSCA','WHCH','WHCD','WHCA','VCCH','VCCD','VCCA','MaxCH',
 'MaxCD','MaxCA','B365C>2.5','B365C<2.5','PC>2.5','PC<2.5','MaxC>2.5','MaxC<2.5','B365CAHH',
 'B365CAHA','PCAHH','PCAHA','MaxCAHH','MaxCAHA',], axis=1, inplace=True)
df.shape

(20921, 40)

### Change Date Format to American

In [77]:
df['Date'] = pd.to_datetime(df['Date'], format="%Y-%m-%d").dt.strftime("%m/%d/%Y")
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HGD,HPts,AGD,APts,HLP,ALP,avgHG,avgAG,avgHST,avgAST
0,E2,08/03/2019,AFC Wimbledon,Rotherham,1,2,A,0.0,1.0,A,...,0.0,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0
1,E2,08/03/2019,Blackpool,Bristol Rvs,2,0,H,1.0,0.0,H,...,0.0,0.0,0.0,0.0,2.0,13.0,0.0,0.0,0.0,0.0
2,E2,08/03/2019,Burton,Ipswich,0,1,A,0.0,1.0,A,...,0.0,0.0,0.0,0.0,3.0,15.0,0.0,0.0,0.0,0.0
3,E2,08/03/2019,Coventry,Southend,1,0,H,0.0,0.0,D,...,0.0,0.0,0.0,0.0,4.0,19.0,0.0,0.0,0.0,0.0
4,E2,08/03/2019,Doncaster,Gillingham,1,1,D,1.0,1.0,D,...,0.0,0.0,0.0,0.0,5.0,14.0,0.0,0.0,0.0,0.0


### Replace Home, Draw, and Away with Numerical Values

In [78]:
# Define a dictionary for the replacements
replacements = {"H": 0, "D": 1, "A": 2}

# Replace the values in the 'FTR' and 'HTR' columns
df['FTR'] = df['FTR'].replace(replacements)
df['HTR'] = df['HTR'].replace(replacements)
df.describe()

,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,...,HGD,HPts,AGD,APts,HLP,ALP,avgHG,avgAG,avgHST,avgAST
count,20921.000000,20921.000000,20921.000000,20920.000000,20920.000000,20920.000000,20920.000000,20920.000000,20920.000000,20920.000000,...,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000,20921.000000
mean,1.447684,1.177190,0.874289,0.647753,0.522897,0.920937,13.133748,11.108270,4.543642,3.812476,...,-0.104536,25.705655,0.119784,25.865494,11.633526,11.608145,1.266114,1.282360,4.038382,4.078137
std,1.236147,1.118807,0.846977,0.812014,0.733635,0.757177,4.995374,4.576118,2.463832,2.236050,...,12.714888,18.202819,12.718642,18.201700,6.179815,6.162287,0.539078,0.554409,1.277342,1.299718
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-61.000000,0.000000,-60.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,8.000000,3.000000,2.000000,...,-7.000000,11.000000,-6.000000,11.000000,6.000000,6.000000,1.000000,1.000000,3.393939,3.416667
50%,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,13.000000,11.000000,4.000000,4.000000,...,0.000000,23.000000,0.000000,23.000000,12.000000,12.000000,1.230769,1.241379,4.000000,4.000000
75%,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,16.000000,14.000000,6.000000,5.000000,...,6.000000,38.000000,6.000000,38.000000,17.000000,17.000000,1.538462,1.550000,4.689655,4.727273
max,9.000000,9.000000,2.000000,7.000000,7.000000,2.000000,46.000000,37.000000,20.000000,20.000000,...,72.000000,98.000000,72.000000,98.000000,24.000000,24.000000,6.000000,8.000000,14.000000,15.000000


### Different Leagues

In [407]:
column_values = df['Div'].unique()
column_values

array(['E2', 'E1', 'E0', 'E3', 'SP2', 'SP1', 'D2', 'D1', 'P1', 'I2', 'I1'],
      dtype=object)

### Check Null Values

In [79]:
# Select the rows with null values
rows_with_null = df[df.isnull().any(axis=1)]

# Print the rows with null values
print(rows_with_null)

     Div        Date      HomeTeam      AwayTeam  FTHG  FTAG  FTR  HTHG  HTAG  \
57    E2  08/31/2019    Gillingham        Bolton     5     0    0   2.0   0.0   
203   E3  11/16/2019  Macclesfield     Mansfield     0     0    1   0.0   0.0   
245   E3  12/14/2019       Walsall  Macclesfield     1     1    1   0.0   1.0   
262   E3  12/26/2019  Macclesfield       Grimsby     1     1    1   0.0   0.0   
291   E3  01/01/2020     Port Vale  Macclesfield     2     2    1   1.0   0.0   
200   E3  11/08/2022     Mansfield      Bradford     1     2    2   0.0   1.0   
76   SP2  09/25/2023      Tenerife       Espanol     1     0    0   1.0   0.0   
50    I2  10/31/2020   Salernitana      Reggiana     3     0    0   NaN   NaN   
371   I2  05/10/2021    Cittadella       Venezia     1     1    1   0.0   1.0   
28    I2  09/03/2023       Brescia       Cosenza     1     0    0   0.0   0.0   
30    I2  09/03/2023         Lecco     Catanzaro     3     4    2   0.0   1.0   
37    I1  10/19/2020        

### NOT APPLIED - Drop columns with more than 60 missing values

In [252]:
# Drop the columns with more than 60 missing values
df = df.dropna(thresh=len(df) - 60, axis=1)

In [64]:
# Count the number of null values in each column
null_counts = df.isnull().sum()

# Filter the counts to include only the ones that are not 0
non_zero_counts = null_counts[null_counts != 0]

# Print the non-zero counts
print(non_zero_counts)

HTHG         1
HTAG         1
HTR          1
HS           1
AS           1
HST          1
AST          1
AvgH        16
AvgD        16
AvgA        16
Avg>2.5     17
Avg<2.5     17
AHh         20
AvgAHH      17
AvgAHA      17
AvgCH        3
AvgCD        3
AvgCA        3
AvgC>2.5     3
AvgC<2.5     3
AHCh         3
AvgCAHH      3
AvgCAHA      3
dtype: int64


### Only 20 rows with null values -> Drop them

In [80]:
df = df.dropna()
df.shape

(20901, 40)

In [81]:
if df.isnull().values.any():
    print("There are null values in df.")
else:
    print("There are no null values in df.")

There are no null values in df.


### Create CSV and Excel Files

In [82]:
# Create an CSV file from df
df.to_csv('clean_data_1.csv', index=False)

# Create an Excel file from df
df.to_excel('clean_data_1.xlsx', index=False)

In [83]:
df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'AvgH', 'AvgD', 'AvgA',
       'Avg>2.5', 'Avg<2.5', 'AHh', 'AvgAHH', 'AvgAHA', 'AvgCH', 'AvgCD',
       'AvgCA', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'AvgCAHH', 'AvgCAHA', 'HGD',
       'HPts', 'AGD', 'APts', 'HLP', 'ALP', 'avgHG', 'avgAG', 'avgHST',
       'avgAST'],
      dtype='object')

In [84]:
df.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'AvgH', 'AvgD', 'AvgA',
       'Avg>2.5', 'Avg<2.5', 'AHh', 'AvgAHH', 'AvgAHA', 'AvgCH', 'AvgCD',
       'AvgCA', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'AvgCAHH', 'AvgCAHA', 'HGD',
       'HPts', 'AGD', 'APts', 'HLP', 'ALP', 'avgHG', 'avgAG', 'avgHST',
       'avgAST'],
      dtype='object')

In [91]:
df_copy = df.copy()
df_clean = df_copy.drop(['Date','FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST','AvgAHH', 'AvgAHA', 'AvgCH', 'AvgCD',
       'AvgCA', 'AvgC>2.5', 'AvgC<2.5', 'AHCh', 'AvgCAHH', 'AvgCAHA', 'AvgCA'], axis=1)
print(df_clean.shape)
df_clean.describe()

(20901, 19)


,AvgH,AvgD,AvgA,Avg>2.5,Avg<2.5,AHh,HGD,HPts,AGD,APts,HLP,ALP,avgHG,avgAG,avgHST,avgAST
count,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000,20901.000000
mean,2.586686,3.668263,3.841576,1.985555,1.908171,-0.253457,-0.100091,25.707765,0.122004,25.866083,11.631118,11.607196,1.266377,1.282354,4.038671,4.078064
std,1.373944,0.878378,2.531073,0.314714,0.367645,0.628231,12.710499,18.202591,12.717589,18.201733,6.179399,6.161546,0.539075,0.554456,1.277275,1.299926
min,1.060000,1.880000,1.080000,1.120000,1.230000,-3.250000,-61.000000,0.000000,-60.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.850000,3.230000,2.510000,1.760000,1.670000,-0.500000,-7.000000,11.000000,-6.000000,11.000000,6.000000,6.000000,1.000000,1.000000,3.393939,3.416667
50%,2.270000,3.430000,3.200000,1.980000,1.820000,-0.250000,0.000000,23.000000,0.000000,23.000000,12.000000,12.000000,1.230769,1.241379,4.000000,4.000000
75%,2.850000,3.760000,4.280000,2.170000,2.050000,0.000000,6.000000,38.000000,6.000000,38.000000,17.000000,17.000000,1.538462,1.550000,4.692308,4.727273
max,34.140000,15.980000,37.570000,3.950000,6.010000,2.750000,72.000000,98.000000,72.000000,98.000000,24.000000,24.000000,6.000000,8.000000,14.000000,15.000000


In [97]:
print(df_clean['Div'].nunique())
print(df_clean['HomeTeam'].nunique())
teams_per_league = df_clean.groupby('Div')['HomeTeam'].nunique()

print("Number of teams in each league:")
print(teams_per_league)

11
272
Number of teams in each league:
Div
D1     24
D2     30
E0     26
E1     39
E2     43
E3     41
I1     29
I2     41
P1     23
SP1    27
SP2    42
Name: HomeTeam, dtype: int64
